In [1]:
import os
import json
import pandas as pd

In [2]:
players=["V Kohli","S Dhawan","RG Sharma","KL Rahul","MS Dhoni","HH Pandya","RA Jadeja","SS Iyer","DA Warner","AJ Finch","SPD Smith","M Labuschagne"]
team=["India","India","India","India","India","India","India","India","Australia","Australia","Australia","Australia"]

In [3]:
path="E:/Cricket Prediction Project/ODI/ODI_MEN"
save="E:/Cricket Prediction Project/ODI/PLAYERS"

In [6]:
for i in range(len(players)):
    data={
        "match_id":[],
        "venue":[],
        "opposition":[],
        "bowler":[],
        "runs_scored":[],
        "balls_faced":[],
        "fours":[],
        "sixes":[]
    }
    set_player_data(players[i],team[i],path,data)
    df=pd.DataFrame(data)
    name=players[i]+".csv"
    df.to_csv(os.path.join(save,name))

OSError: Cannot save file into a non-existent directory: 'E:\Cricket Prediction Project\ODI\PLAYERS'

In [5]:
def set_player_data(player_name,team,path,data):
    for filename in os.listdir(path):
        name_parts=str(filename).split()
        match_id=name_parts[-1].split(".")[0]
        inning=""
        file_path=os.path.join(path,filename)
        df_1=pd.read_csv(os.path.join(file_path,"1st_Innings.csv"))
        df_2=pd.read_csv(os.path.join(file_path,"2nd_Innings.csv"))
        meta_data=json.load(open(os.path.join(file_path,"meta_data.json")))
        #print(meta_data["teams"])
        if( not (team in meta_data["teams"])):
            continue
        df=pd.DataFrame()
        if(player_name in list(df_1["batter"])):
            df=pd.DataFrame(df_1)
        elif(player_name in list(df_2["batter"])):
            df=pd.DataFrame(df_2)
        else:
            continue
        data_match={
            "match_id":[],
            "venue":[],
            "opposition":[],
            "bowler":[],
            "runs_scored":[],
            "balls_faced":[],
            "fours":[],
            "sixes":[]
        }
        opposition=meta_data["teams"][1] if meta_data["teams"][0]==team else meta_data["teams"][0]
        df_player=pd.DataFrame(df.loc[df["batter"]==player_name])
        bowler_set=set(list(df_player["bowler"]))
        for i in bowler_set:
            data_match["venue"].append(meta_data["venue"])
            data_match["opposition"].append(opposition)
            data_match["match_id"].append(match_id)
            data_match["bowler"].append(i)
            df_player_bowler=df_player.loc[df_player['bowler']==i]
            data_match["runs_scored"].append(sum(list(df_player_bowler["run_hit"])))
            data_match["balls_faced"].append(len(df_player_bowler))
            data_match["fours"].append(len(df_player_bowler.loc[df_player_bowler['run_hit']==4]))
            data_match["sixes"].append(len(df_player_bowler.loc[df_player_bowler['run_hit']==6]))
            attributes=["match_id",
            "venue",
            "opposition",
            "bowler",
            "runs_scored",
            "balls_faced",
            "fours",
            "sixes"]
        for i in attributes:
            data[i]=data[i]+data_match[i]